In [8]:
!pip install transformers datasets accelerate

In [9]:
from datasets import Dataset

# Sample Q&A data (your own domain-specific examples)
data = {
    "question": [
        "What is LangChain?",
        "What is FAISS used for?",
        "How does RAG work?",
        "What is GPT-4?",
    ],
    "answer": [
        "LangChain is a framework to build applications with LLMs.",
        "FAISS is a library for fast similarity search using embeddings.",
        "RAG combines retrieval with LLMs to generate grounded responses.",
        "GPT-4 is a large language model developed by OpenAI.",
    ]
}

dataset = Dataset.from_dict(data)
dataset = dataset.train_test_split(test_size=0.2)


In [19]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments

# Sample data
data = {
    "question": [
        "What is LangChain?",
        "What is FAISS used for?",
    ],
    "answer": [
        "LangChain is a framework to build LLM applications.",
        "FAISS is used for fast similarity search using embeddings.",
    ]
}

dataset = Dataset.from_dict(data).train_test_split(test_size=0.2)

# Load tokenizer & model
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def preprocess(batch):
    inputs = tokenizer(["question: " + q for q in batch["question"]],
                       padding="max_length", truncation=True, max_length=128)
    targets = tokenizer(batch["answer"],
                        padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized = dataset.map(preprocess, batched=True)

# Training args
args = Seq2SeqTrainingArguments(
    output_dir="./t5-finetuned",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=1,
    #evaluation_strategy="epoch"
)

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer
)

# Train
trainer.train()


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-19-11821d0d44bf>:46: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss


TrainOutput(global_step=3, training_loss=31.583651224772137, metrics={'train_runtime': 17.8969, 'train_samples_per_second': 0.168, 'train_steps_per_second': 0.168, 'total_flos': 139417878528.0, 'train_loss': 31.583651224772137, 'epoch': 3.0})